# Predicting the Superbowl LIX Winner!

In [403]:
import pandas as pd
import numpy as np
import io

https://www.pro-football-reference.com/years/2024/index.htm

Pulling stats on the top 8 teams each season, we are going to use this data to train a model to predict the Superbowl LIX winner. The idea is that the model will draw correlations between certain features and a teams given ranking that season. Once the model has ranked the top 8 teams this 2024 season -- Houston Texans, Kansas City Chiefs, Baltimore Ravens, Buffalo Bills, Washington Commanders, Detriot Lions, Los Angeles Rams, Philadelphia Eagles -- we can use those predicted rankings to determine the outcomes of the remainder of the playoffs bracket.

We will be looking at regular season results between 2000 and 2023.

## Data Collection

The above website as editable tables for each years regular season results, so after selecting the top 8 teams of each given year and the desired features, we can paste the CSV for that year in. This is done separately for offense and defense.

In [404]:
csv_00_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Oakland Raiders,479,5.6,20,9,337,11,6.6,4.8,39.2,10.6
Minnesota Vikings,397,6.2,28,10,319,18,7.2,5.0,39.7,14.9
New Orleans Saints,354,5.2,26,11,312,15,6.2,4.1,32.4,12.6
Philadelphia Eagles,351,4.9,29,14,295,15,5.0,4.7,33.3,13.8
Tennessee Titans,346,5.2,30,14,299,16,6.7,3.8,30.4,14.9
Baltimore Ravens,333,4.7,26,7,288,19,5.1,4.3,32.8,12.8
New York Giants,328,5.1,24,11,310,13,6.0,4.0,29.7,12.0
Miami Dolphins,323,4.7,26,9,251,17,5.7,3.8,30.6,13.0"""

csv_00_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Baltimore Ravens,165,4.3,49,26,216,23,5.3,2.7,15.5,23.7
Tennessee Titans,191,4.2,30,13,215,17,4.7,3.6,18.1,13.8
Miami Dolphins,226,4.7,41,13,289,28,5.0,4.2,22.8,19.7
Philadelphia Eagles,245,4.6,31,12,295,19,5.0,4.0,25.1,14.9
New York Giants,246,4.6,31,11,274,20,5.4,3.2,22.6,15.3
Oakland Raiders,299,5.2,37,16,320,21,5.9,4.0,26.9,18.7
New Orleans Saints,305,5.0,35,15,279,20,5.5,4.2,27.7,17.5
Minnesota Vikings,371,5.6,18,10,344,8,6.4,4.5,38.2,9.4"""

csv_01_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
St. Louis Rams,503,6.6,44,22,357,22,7.9,4.9,44.0,20.9
Oakland Raiders,399,5.2,25,16,316,9,6.4,3.7,36.2,10.8
Green Bay Packers,390,5.8,27,12,282,15,7.1,4.1,32.3,12.8
New England Patriots,371,4.9,28,13,292,15,5.9,3.8,32.4,14.6
Pittsburgh Steelers,352,5.5,21,9,314,12,6.4,4.8,34.6,11.5
Philadelphia Eagles,343,5.1,24,10,256,14,5.6,4.3,29.4,10.9
Chicago Bears,338,4.6,24,8,277,16,5.4,3.7,29.7,11.3
Baltimore Ravens,303,4.7,36,16,299,20,5.6,3.7,29.2,16.3"""

csv_01_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Chicago Bears,203,4.9,37,17,277,20,5.6,3.5,18.7,18.7
Philadelphia Eagles,208,4.6,33,19,262,14,5.1,4.0,21.1,14.1
Pittsburgh Steelers,212,4.5,28,12,254,16,5.1,3.5,18.5,13.5
Baltimore Ravens,265,4.4,28,12,262,16,5.1,3.4,22.1,13.2
Green Bay Packers,266,4.7,39,19,278,20,5.0,4.4,23.7,17.5
New England Patriots,272,5.3,35,13,303,22,6.0,4.3,27.7,17.6
St. Louis Rams,273,4.7,34,13,256,21,5.3,3.8,25.5,18.1
Oakland Raiders,327,5.1,24,7,296,17,5.4,4.6,30.4,12.0"""

csv_02_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Oakland Raiders,450,5.8,19,9,366,10,6.8,4.3,42.7,9.4
Philadelphia Eagles,415,5.2,24,13,311,11,5.8,4.5,36.0,12.0
Atlanta Falcons,402,5.3,27,15,316,12,6.1,4.5,38.6,13.2
Pittsburgh Steelers,390,5.4,36,14,343,22,6.6,4.1,35.7,16.8
San Francisco 49ers,367,5.3,17,7,342,10,5.8,4.6,37.6,9.2
Tennessee Titans,367,5.1,25,10,312,15,6.4,3.8,34.6,13.2
New York Jets,359,5.5,19,9,300,10,6.6,4.0,36.1,10.7
Tampa Bay Buccaneers,346,4.9,21,11,287,10,5.7,3.8,32.6,10.7"""

csv_02_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Tampa Bay Buccaneers,196,4.2,38,7,236,31,4.5,3.8,17.8,18.3
Philadelphia Eagles,241,4.7,38,23,272,15,5.0,4.3,20.6,18.0
Oakland Raiders,304,5.0,31,10,319,21,5.8,3.8,28.7,16.6
Atlanta Falcons,314,5.3,39,15,288,24,5.9,4.6,27.9,19.5
Tennessee Titans,324,5.1,29,11,297,18,5.9,3.8,27.1,14.7
New York Jets,336,5.4,23,8,313,15,6.0,4.6,36.4,12.7
Pittsburgh Steelers,345,4.9,36,17,279,19,5.6,3.8,29.8,16.5
San Francisco 49ers,351,5.3,27,8,307,19,6.0,4.2,34.9,14.8"""

csv_03_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Kansas City Chiefs,484,5.9,18,6,348,12,7.1,4.3,38.3,8.5
St. Louis Rams,447,5.2,39,16,335,23,6.2,3.6,41.1,16.8
Indianapolis Colts,447,5.6,20,10,348,10,7.1,3.7,46.3,10.2
Green Bay Packers,442,5.8,32,11,315,21,6.6,5.0,38.0,15.6
Tennessee Titans,435,5.4,21,12,310,9,7.4,3.3,39.4,9.6
Philadelphia Eagles,374,5.3,22,11,302,11,5.7,4.8,34.2,10.2
New England Patriots,348,4.8,24,11,294,13,6.0,3.4,27.9,11.3
Carolina Panthers,325,5.1,31,15,284,16,6.3,4.0,30.8,14.9"""

csv_03_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,238,4.4,41,12,293,29,4.9,3.6,23.0,19.5
Philadelphia Eagles,287,5.0,26,13,306,13,5.4,4.5,27.8,13.4
Carolina Panthers,304,4.7,26,10,274,16,5.3,4.0,26.3,12.6
Green Bay Packers,307,4.9,32,11,288,21,5.5,4.1,29.6,15.3
Tennessee Titans,324,5.3,34,13,275,21,6.2,3.8,29.8,16.0
St. Louis Rams,328,5.2,46,22,272,24,5.6,4.8,26.3,22.2
Kansas City Chiefs,332,5.4,37,12,322,25,5.6,5.2,29.5,17.4
Indianapolis Colts,336,5.2,30,15,279,15,5.9,4.5,30.1,14.8"""

csv_04_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Indianapolis Colts,522,6.7,17,7,379,10,8.5,4.3,46.3,9.7
New England Patriots,437,5.5,27,13,344,14,7.0,4.1,42.1,14.6
Minnesota Vikings,405,6.4,21,9,351,12,7.6,4.7,38.9,12.0
Philadelphia Eagles,386,5.9,22,11,301,11,6.8,4.4,35.9,10.9
Pittsburgh Steelers,372,5.1,21,8,309,13,6.9,4.0,35.8,11.7
Atlanta Falcons,340,5.2,30,14,284,16,5.4,5.1,28.8,15.8
New York Jets,333,5.5,16,5,313,11,6.5,4.5,33.3,8.6
St. Louis Rams,319,5.8,39,17,321,22,6.8,4.3,30.3,20.6"""

csv_04_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Pittsburgh Steelers,251,4.7,32,13,248,19,5.4,3.6,25.9,17.2
Philadelphia Eagles,260,4.9,28,11,299,17,5.4,4.3,25.1,13.1
New England Patriots,260,5.0,36,16,290,20,5.8,3.9,24.0,20.0
New York Jets,261,5.0,33,14,282,19,6.2,3.6,25.3,16.9
Atlanta Falcons,337,5.2,32,13,310,19,6.2,3.9,29.7,16.8
Indianapolis Colts,351,5.7,36,17,331,19,6.5,4.6,35.4,18.5
St. Louis Rams,392,5.3,15,9,311,6,6.0,4.5,37.6,7.7
Minnesota Vikings,395,5.8,22,11,350,11,6.7,4.6,41.6,13.3"""

csv_05_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Seattle Seahawks,452,5.8,17,7,361,10,6.9,4.7,38.9,8.1
Indianapolis Colts,439,5.8,19,8,363,11,7.7,3.7,43.6,9.1
Denver Broncos,395,5.6,16,9,330,7,6.6,4.7,36.0,8.1
Carolina Panthers,391,5.1,26,10,278,16,6.9,3.4,35.8,12.6
Pittsburgh Steelers,389,5.4,23,9,297,14,7.1,4.0,36.1,10.9
New England Patriots,379,5.5,24,9,334,15,7.0,3.4,34.8,13.0
Washington Redskins,359,5.1,27,16,301,11,6.1,4.2,29.7,13.0
Chicago Bears,260,4.4,28,13,233,15,4.5,4.3,22.6,12.8"""

csv_05_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Chicago Bears,202,4.4,34,10,259,24,4.9,3.7,19.7,15.3
Indianapolis Colts,247,5.2,31,13,269,18,5.7,4.4,26.9,18.0
Pittsburgh Steelers,258,4.6,30,15,275,15,5.3,3.4,27.1,14.9
Denver Broncos,258,5.1,36,16,295,20,5.7,4.0,25.0,18.2
Carolina Panthers,259,4.6,42,19,262,23,5.3,3.6,25.1,19.8
Seattle Seahawks,271,4.9,27,11,295,16,5.7,3.6,30.5,12.8
Washington Redskins,293,4.9,28,12,258,16,5.4,4.1,26.5,13.8
New England Patriots,338,5.3,18,8,306,10,6.6,3.6,33.0,9.3"""

csv_06_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
San Diego Chargers,492,5.7,15,6,321,9,6.6,4.9,43.2,6.8
Chicago Bears,427,5.0,36,14,300,22,6.1,3.8,34.8,15.9
Indianapolis Colts,427,6.0,19,10,376,9,7.5,4.0,47.4,10.9
New Orleans Saints,413,5.8,23,10,330,13,7.5,3.7,36.8,12.1
Philadelphia Eagles,398,6.2,24,15,312,9,7.2,4.8,33.9,11.5
New England Patriots,385,5.1,27,15,330,12,6.1,3.9,34.4,13.8
Baltimore Ravens,353,5.0,23,9,282,14,6.3,3.4,31.3,11.5
Seattle Seahawks,335,4.8,34,12,312,22,5.4,4.0,29.9,17.3"""

csv_06_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Baltimore Ravens,201,4.5,40,12,236,28,5.3,3.3,19.5,20.0
New England Patriots,237,5.0,35,13,264,22,5.7,3.9,23.6,18.7
Chicago Bears,255,4.6,44,20,258,24,5.0,4.0,20.6,19.6
San Diego Chargers,303,4.9,28,12,285,16,5.4,4.2,28.8,13.6
New Orleans Saints,322,5.3,19,8,262,11,5.6,4.9,26.5,9.0
Philadelphia Eagles,328,5.0,29,10,314,19,5.4,4.5,30.1,15.1
Seattle Seahawks,341,5.4,26,14,277,12,6.0,4.6,29.6,11.7
Indianapolis Colts,360,5.5,26,11,325,15,5.8,5.3,38.5,16.7"""

csv_07_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,589,6.2,15,6,393,9,7.8,4.1,52.7,7.8
Dallas Cowboys,455,6.0,24,5,322,19,7.4,4.2,41.8,12.6
Indianapolis Colts,450,5.6,19,5,357,14,7.0,3.8,44.6,11.4
Green Bay Packers,435,6.0,24,9,307,15,7.3,4.1,38.9,12.1
San Diego Chargers,412,5.1,24,8,278,16,6.1,4.2,34.0,12.6
Jacksonville Jaguars,411,5.6,21,13,328,8,6.7,4.6,39.6,10.1
Seattle Seahawks,393,5.3,24,11,323,13,6.3,3.8,33.2,11.4
New York Giants,373,5.1,34,14,321,20,5.5,4.6,32.3,17.5"""

csv_07_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Indianapolis Colts,262,4.6,37,15,288,22,5.3,3.8,25.3,21.6
New England Patriots,274,4.9,31,12,278,19,5.3,4.4,24.0,16.6
San Diego Chargers,284,5.1,48,18,297,30,5.7,4.1,23.8,22.7
Seattle Seahawks,291,5.0,34,14,278,20,5.7,3.9,25.5,15.9
Green Bay Packers,291,5.0,28,9,297,19,5.9,3.9,29.1,13.2
Jacksonville Jaguars,304,5.2,30,10,286,20,5.9,4.1,29.2,17.0
Dallas Cowboys,325,4.9,29,10,304,19,5.4,4.0,30.8,14.8
New York Giants,351,5.0,25,10,288,15,5.8,3.8,29.5,12.4"""

csv_08_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
San Diego Chargers,439,6.0,20,9,301,11,7.7,4.1,43.5,10.6
New York Giants,427,5.6,13,3,338,10,6.1,5.0,43.8,7.3
Arizona Cardinals,427,5.9,30,15,328,15,7.1,3.5,39.3,15.7
Philadelphia Eagles,416,5.3,26,10,318,16,6.2,4.0,35.5,11.5
Carolina Panthers,414,6.0,19,7,287,12,7.3,4.8,38.0,9.9
Baltimore Ravens,385,4.9,21,9,300,12,6.0,4.0,33.0,10.8
Tennessee Titans,375,5.2,17,8,268,9,6.1,4.3,34.6,8.9
Pittsburgh Steelers,347,4.9,25,10,290,15,5.9,3.7,32.6,12.6"""

csv_08_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Pittsburgh Steelers,223,3.9,29,9,240,20,4.3,3.3,22.9,14.4
Tennessee Titans,234,4.6,31,11,276,20,5.2,3.7,23.2,15.5
Baltimore Ravens,244,4.5,34,8,228,26,5.1,3.6,21.5,17.4
Philadelphia Eagles,289,4.4,29,14,248,15,5.1,3.5,25.3,13.6
New York Giants,294,5.0,22,5,268,17,5.8,4.0,28.9,12.1
Carolina Panthers,329,5.2,25,13,296,12,5.7,4.4,30.8,12.4
San Diego Chargers,347,5.4,24,9,339,15,6.3,4.0,35.5,13.0
Arizona Cardinals,426,5.3,30,17,312,13,6.5,4.0,38.7,15.5"""

csv_09_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New Orleans Saints,510,6.3,28,16,348,12,7.7,4.5,41.6,13.5
Minnesota Vikings,470,5.8,18,11,343,7,7.1,4.1,42.0,9.0
San Diego Chargers,454,5.9,17,7,330,10,8.0,3.3,45.9,9.4
Indianapolis Colts,416,5.9,24,5,339,19,7.4,3.5,37.9,13.2
Baltimore Ravens,391,5.5,22,9,320,13,6.3,4.7,35.0,10.9
Arizona Cardinals,375,5.6,36,18,317,18,6.5,4.1,30.7,17.1
Dallas Cowboys,361,6.3,19,10,335,9,7.3,4.8,33.5,9.5
New York Jets,348,5.0,30,9,280,21,5.6,4.5,31.8,15.2"""

csv_09_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New York Jets,236,4.2,31,14,237,17,4.6,3.8,20.3,14.4
Dallas Cowboys,250,5.2,21,10,286,11,5.9,4.0,24.3,11.6
Baltimore Ravens,261,4.9,32,10,280,22,6.0,3.4,26.3,16.7
Indianapolis Colts,307,5.0,26,10,320,16,5.5,4.3,33.0,14.0
Minnesota Vikings,312,5.2,24,13,271,11,6.0,3.9,28.2,11.7
San Diego Chargers,320,5.3,25,11,309,14,5.9,4.5,34.5,14.3
Arizona Cardinals,325,5.3,29,8,289,21,5.9,4.5,27.6,13.8
New Orleans Saints,341,5.5,39,13,310,26,6.2,4.5,31.4,18.0"""

csv_10_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,518,5.9,10,5,335,5,7.2,4.3,47.0,5.4
Atlanta Falcons,414,5.0,17,8,353,9,5.9,3.8,39.3,9.0
Green Bay Packers,388,5.7,22,9,312,13,7.1,3.8,35.0,10.9
Pittsburgh Steelers,375,5.6,18,9,295,9,6.9,4.1,36.9,9.5
New York Jets,367,5.2,21,7,307,14,5.9,4.4,32.2,10.6
Baltimore Ravens,357,5.1,20,10,303,10,6.3,3.8,32.8,9.5
Chicago Bears,334,4.9,31,10,263,21,5.8,3.9,29.3,15.7
Seattle Seahawks,310,4.9,31,11,260,20,5.8,3.7,26.5,14.8"""

csv_10_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Pittsburgh Steelers,232,4.5,35,14,272,21,5.3,3.0,25.7,17.3
Green Bay Packers,240,5.1,32,8,270,24,5.4,4.7,25.3,16.5
Baltimore Ravens,270,5.1,27,8,284,19,5.8,3.9,29.2,13.5
Chicago Bears,286,5.0,35,14,282,21,5.8,3.7,25.0,17.9
Atlanta Falcons,288,5.6,31,9,284,22,6.1,4.6,26.1,16.1
New York Jets,304,4.8,30,18,272,12,5.6,3.6,26.4,13.9
New England Patriots,313,5.6,38,13,348,25,6.4,4.2,30.7,21.0
Seattle Seahawks,407,5.5,22,10,313,12,6.4,4.2,35.9,9.9"""

csv_11_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Green Bay Packers,560,6.6,14,6,353,8,8.3,3.9,48.3,6.1
New Orleans Saints,547,6.7,19,5,416,14,7.8,4.9,51.4,10.9
New England Patriots,513,6.3,17,5,399,12,7.9,4.0,47.2,8.3
New York Giants,394,6.0,24,8,331,16,7.7,3.5,32.5,11.5
Houston Texans,381,5.7,20,11,321,9,7.0,4.5,37.2,10.5
San Francisco 49ers,380,5.0,10,5,282,5,5.9,4.1,39.6,5.2
Baltimore Ravens,378,5.2,24,12,312,12,5.9,4.3,35.5,11.8
Denver Broncos,309,5.0,30,17,286,13,5.2,4.8,24.5,14.2"""

csv_11_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
San Francisco 49ers,229,5.1,38,15,267,23,6.0,3.5,23.3,18.7
Baltimore Ravens,266,4.6,26,11,269,15,5.4,3.5,26.6,13.8
Houston Texans,278,4.8,27,10,270,17,5.2,4.1,29.8,13.8
New Orleans Saints,339,5.8,16,7,326,9,6.3,5.0,34.1,8.8
New England Patriots,342,6.2,34,11,370,23,7.1,4.6,32.6,17.1
Green Bay Packers,359,6.3,38,7,358,31,7.2,4.7,33.9,20.2
Denver Broncos,390,5.4,18,9,315,9,6.4,4.1,33.0,7.8
New York Giants,400,5.6,31,11,338,20,6.4,4.5,34.7,14.8"""

csv_12_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,557,5.7,16,7,444,9,7.0,4.2,48.1,8.1
Denver Broncos,481,5.8,25,14,380,11,7.4,3.8,39.9,12.2
Green Bay Packers,433,5.5,16,8,341,8,6.6,3.9,37.6,8.1
Atlanta Falcons,419,5.8,18,4,343,14,7.0,3.7,44.0,10.3
Houston Texans,416,5.5,17,4,341,13,6.6,4.2,35.6,8.4
Seattle Seahawks,412,5.8,18,8,311,10,6.9,4.8,39.4,10.0
Baltimore Ravens,398,5.4,16,5,314,11,6.3,4.3,35.8,7.8
San Francisco 49ers,397,6.0,16,8,322,8,6.9,5.1,39.3,7.9"""

csv_12_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Seattle Seahawks,245,5.1,31,13,295,18,5.4,4.5,27.2,16.0
San Francisco 49ers,273,4.7,25,11,284,14,5.3,3.7,26.7,12.8
Denver Broncos,289,4.6,24,8,287,16,5.2,3.6,26.9,11.9
Atlanta Falcons,299,5.9,31,11,298,20,6.7,4.8,31.7,17.2
Houston Texans,331,5.1,29,14,292,15,5.8,4.0,28.0,14.5
New England Patriots,331,5.7,41,21,340,20,6.9,3.9,30.1,20.4
Green Bay Packers,336,5.2,23,5,308,18,5.7,4.5,33.2,11.8
Baltimore Ravens,344,5.2,25,12,326,13,6.1,4.0,34.9,12.0"""

csv_13_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Denver Broncos,606,6.3,26,16,435,10,7.8,4.1,47.5,10.4
New England Patriots,444,5.4,20,9,378,11,6.1,4.4,40.8,10.0
Seattle Seahawks,417,5.6,19,10,307,9,7.0,4.3,40.7,9.3
New Orleans Saints,414,5.9,19,7,359,12,7.1,3.8,40.3,8.3
San Francisco 49ers,406,5.4,18,10,286,8,6.5,4.4,37.8,9.0
San Diego Chargers,396,5.9,21,10,373,11,7.5,4.0,44.9,11.4
Indianapolis Colts,391,5.3,14,4,318,10,6.1,4.3,38.8,6.9
Carolina Panthers,366,5.1,19,6,319,13,5.9,4.2,36.5,9.4"""

csv_13_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Seattle Seahawks,231,4.4,39,11,282,28,4.8,3.9,24.7,19.4
Carolina Panthers,241,4.9,30,10,278,20,5.5,4.0,28.7,16.3
San Francisco 49ers,272,5.0,30,12,284,18,5.7,3.9,26.2,13.1
New Orleans Saints,304,5.2,19,7,274,12,5.6,4.6,30.4,10.5
Indianapolis Colts,336,5.6,27,12,319,15,6.4,4.5,31.7,13.7
New England Patriots,338,5.3,29,12,338,17,6.0,4.5,29.6,13.3
San Diego Chargers,348,6.1,17,6,327,11,7.1,4.6,36.1,9.6
Denver Broncos,399,5.3,26,9,339,17,6.2,3.9,32.5,12.6"""

csv_14_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Green Bay Packers,486,6.2,13,7,356,6,7.5,4.4,46.7,7.1
Denver Broncos,482,6.0,20,5,360,15,7.5,4.0,39.9,9.6
New England Patriots,468,5.5,13,4,361,9,6.5,3.9,45.3,7.2
Dallas Cowboys,467,6.1,25,14,342,11,7.5,4.6,43.6,12.8
Indianapolis Colts,458,5.9,31,15,371,16,7.1,3.9,39.5,14.6
Baltimore Ravens,409,5.7,20,8,345,12,6.7,4.5,39.6,9.9
Seattle Seahawks,394,5.9,14,7,328,7,6.6,5.3,40.6,6.9
Carolina Panthers,339,5.2,23,11,347,12,6.0,4.3,34.8,11.8"""

csv_14_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Seattle Seahawks,254,4.6,24,11,277,13,5.5,3.4,26.9,12.3
Baltimore Ravens,302,5.2,22,11,311,11,6.2,3.6,33.3,11.7
New England Patriots,313,5.3,25,9,329,16,6.2,4.0,32.4,13.2
Green Bay Packers,348,5.3,27,9,339,18,6.0,4.3,33.1,14.9
Dallas Cowboys,352,5.8,31,13,315,18,6.9,4.2,31.3,16.8
Denver Broncos,354,4.7,25,7,311,18,5.3,3.7,31.3,11.8
Indianapolis Colts,369,5.4,26,14,309,12,6.3,4.3,28.1,12.6
Carolina Panthers,374,5.4,26,12,309,14,6.1,4.5,34.1,14.0"""

csv_15_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Carolina Panthers,500,5.5,19,9,357,10,6.7,4.3,42.9,9.6
Arizona Cardinals,489,6.3,24,11,373,13,7.8,4.2,42.5,11.8
New England Patriots,465,5.7,14,7,348,7,6.9,3.7,43.2,5.7
Seattle Seahawks,423,5.9,16,8,335,8,7.1,4.5,42.0,8.6
Pittsburgh Steelers,423,6.3,28,7,331,21,7.4,4.4,40.5,13.7
Kansas City Chiefs,405,5.5,15,8,303,7,6.3,4.7,37.9,7.1
Green Bay Packers,368,5.1,17,9,317,8,5.7,4.2,33.3,8.9
Denver Broncos,355,5.4,31,8,314,23,6.2,4.2,31.2,15.1"""

csv_15_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Seattle Seahawks,277,4.9,23,9,273,14,5.8,3.6,29.3,13.2
Kansas City Chiefs,287,5.1,29,7,313,22,5.7,4.1,27.3,15.3
Denver Broncos,296,4.4,27,13,289,14,5.1,3.3,26.9,11.9
Carolina Panthers,308,4.9,39,15,298,24,5.4,3.9,26.5,19.4
Arizona Cardinals,313,5.2,33,14,284,19,6.1,3.9,31.7,16.7
New England Patriots,315,5.2,21,9,303,12,6.0,4.0,28.9,10.5
Pittsburgh Steelers,319,5.5,30,13,327,17,6.5,3.8,32.3,14.3
Green Bay Packers,323,5.5,22,6,300,16,6.1,4.5,31.3,11.5"""

csv_16_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,441,5.9,11,9,351,2,7.5,3.9,42.9,4.9
Green Bay Packers,432,5.7,17,9,349,8,6.4,4.5,43.8,9.1
Dallas Cowboys,421,6.0,15,9,358,6,7.1,4.8,43.9,8.7
Arizona Cardinals,418,5.4,28,11,368,17,6.0,4.3,35.0,14.2
Pittsburgh Steelers,399,5.8,18,3,336,15,6.8,4.3,38.5,9.9
Kansas City Chiefs,389,5.5,17,9,305,8,6.5,4.2,37.1,9.0
Seattle Seahawks,354,5.6,18,6,313,12,6.8,3.9,37.3,9.7
Houston Texans,279,4.7,24,8,305,16,5.2,4.1,31.4,11.4"""

csv_16_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,250,5.2,23,10,294,13,6.0,3.9,26.8,11.2
Seattle Seahawks,292,5.0,19,8,307,11,6.2,3.4,28.3,10.3
Dallas Cowboys,306,5.5,20,11,330,9,6.2,3.9,35.1,11.3
Kansas City Chiefs,311,5.5,33,15,342,18,6.3,4.4,31.9,16.5
Pittsburgh Steelers,327,5.5,23,10,313,13,6.2,4.3,34.5,11.9
Houston Texans,328,5.1,17,6,272,11,5.8,4.0,31.7,9.1
Arizona Cardinals,362,4.8,28,14,293,14,5.7,3.6,32.5,12.9
Green Bay Packers,388,5.9,25,8,329,17,7.1,4.0,41.0,13.9"""

csv_17_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
New England Patriots,458,5.9,12,4,389,8,7.1,4.2,49.4,6.9
Philadelphia Eagles,457,5.5,20,11,338,9,6.2,4.5,40.3,9.4
New Orleans Saints,448,6.3,18,10,332,8,7.5,4.7,44.6,9.1
Jacksonville Jaguars,417,5.4,23,10,332,13,6.5,4.3,34.7,11.1
Pittsburgh Steelers,406,5.8,20,5,348,15,7.1,3.8,43.0,10.6
Minnesota Vikings,382,5.4,14,6,334,8,6.8,3.9,38.9,7.6
Atlanta Falcons,353,5.9,18,6,330,12,7.2,4.3,41.2,10.3
Tennessee Titans,334,5.2,25,8,290,17,6.0,4.1,35.1,12.6"""

csv_17_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Minnesota Vikings,252,4.6,19,5,260,14,5.2,3.7,27.9,10.1
Jacksonville Jaguars,268,4.6,33,12,257,21,4.8,4.3,23.9,15.6
Philadelphia Eagles,295,5.0,31,12,272,19,5.7,3.8,27.1,15.4
New England Patriots,296,5.7,18,6,325,12,6.4,4.7,30.2,9.9
Pittsburgh Steelers,308,5.2,22,6,270,16,5.8,4.4,30.3,12.4
Atlanta Falcons,315,5.1,16,8,323,8,5.8,4.1,36.0,9.1
New Orleans Saints,326,5.4,25,5,308,20,6.0,4.4,33.3,13.6
Tennessee Titans,356,5.0,21,9,323,12,5.9,3.6,37.3,10.8"""

csv_18_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Kansas City Chiefs,565,6.8,18,6,384,12,8.1,4.8,52.6,10.5
Los Angeles Rams,527,6.4,19,7,401,12,7.5,4.9,48.6,9.8
New Orleans Saints,504,6.0,16,9,377,7,7.5,4.3,51.8,9.5
New England Patriots,436,5.9,18,7,365,11,7.2,4.3,40.8,9.2
Indianapolis Colts,433,5.8,24,9,371,15,6.7,4.2,41.2,13.2
Los Angeles Chargers,428,6.3,19,7,338,12,7.5,4.7,42.4,10.0
Philadelphia Eagles,367,5.6,23,12,346,11,6.7,3.9,38.1,12.5
Dallas Cowboys,339,5.4,17,9,322,8,6.1,4.5,37.9,10.1"""

csv_18_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Dallas Cowboys,324,5.4,20,11,309,9,6.5,3.8,36.5,12.0
New England Patriots,325,5.7,28,10,322,18,6.2,4.9,32.6,15.0
Los Angeles Chargers,329,5.4,20,7,311,13,6.2,4.3,33.7,11.8
Indianapolis Colts,344,5.5,26,11,315,15,6.6,3.9,35.5,14.5
Philadelphia Eagles,348,5.8,17,7,308,10,6.4,4.7,38.6,9.1
New Orleans Saints,353,5.7,24,12,333,12,6.9,3.6,36.0,14.6
Los Angeles Rams,384,6.0,30,12,322,18,6.6,5.1,38.2,16.9
Kansas City Chiefs,421,5.9,27,12,419,15,6.4,5.0,40.8,14.9"""

csv_19_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Baltimore Ravens,531,6.1,15,7,386,8,6.9,5.5,51.8,8.4
San Francisco 49ers,479,6.0,23,10,336,13,7.4,4.6,44.3,12.0
Kansas City Chiefs,451,6.2,15,10,350,5,7.5,4.2,48.8,7.9
Minnesota Vikings,407,5.8,20,12,314,8,7.1,4.5,41.6,10.4
Seattle Seahawks,405,5.7,20,14,341,6,6.7,4.6,36.5,10.6
Tennessee Titans,402,6.1,17,9,317,8,7.1,5.0,31.4,8.6
Houston Texans,378,5.7,22,8,346,14,6.5,4.6,37.9,12.4
Green Bay Packers,376,5.4,13,9,320,4,6.1,4.4,37.3,6.8"""

csv_19_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Baltimore Ravens,282,5.2,25,12,276,13,5.7,4.4,32.5,14.5
Minnesota Vikings,303,5.2,31,14,324,17,5.8,4.3,34.7,17.3
Kansas City Chiefs,308,5.4,23,7,344,16,5.7,4.9,34.1,13.4
San Francisco 49ers,310,4.7,27,15,285,12,4.8,4.5,28.8,14.1
Green Bay Packers,313,5.7,25,8,310,17,6.3,4.7,34.3,14.0
Tennessee Titans,331,5.4,23,9,335,14,6.4,4.0,31.7,11.6
Houston Texans,385,6.1,22,10,346,12,6.9,4.8,38.8,12.9
Seattle Seahawks,398,6.0,32,16,346,16,6.7,4.9,36.9,16.2"""

csv_20_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Green Bay Packers,509,6.3,11,6,358,5,7.5,4.8,49.7,5.6
Buffalo Bills,501,6.1,22,11,397,11,7.4,4.2,49.4,11.8
Tampa Bay Buccaneers,492,6.0,17,5,364,12,7.1,4.1,47.8,8.9
New Orleans Saints,482,5.8,17,9,367,8,6.8,4.6,45.5,9.0
Kansas City Chiefs,473,6.3,16,9,397,7,7.4,4.5,47.9,9.2
Baltimore Ravens,468,5.9,18,7,327,11,6.3,5.5,45.3,10.6
Cleveland Browns,408,5.8,16,8,355,8,6.7,4.8,40.6,9.7
Los Angeles Rams,372,5.5,25,11,352,14,6.5,4.3,34.2,12.5"""

csv_20_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Los Angeles Rams,296,4.6,22,8,280,14,5.1,3.8,27.9,12.0
Baltimore Ravens,303,5.2,22,12,347,10,5.6,4.6,30.0,12.4
New Orleans Saints,337,5.0,26,8,320,18,5.8,3.9,34.7,13.1
Tampa Bay Buccaneers,355,5.1,25,10,319,15,5.9,3.6,33.7,14.0
Kansas City Chiefs,362,5.6,22,6,353,16,6.4,4.5,36.1,13.0
Green Bay Packers,369,5.5,18,7,332,11,6.1,4.5,39.9,11.4
Buffalo Bills,375,5.5,26,11,351,15,6.1,4.6,39.4,15.3
Cleveland Browns,419,5.5,21,10,352,11,6.4,4.3,43.0,12.7"""

csv_21_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Tampa Bay Buccaneers,511,6.1,19,7,404,12,6.9,4.3,44.3,9.8
Buffalo Bills,483,5.7,22,6,398,16,6.3,4.8,45.2,11.3
Kansas City Chiefs,480,5.9,25,12,419,13,6.8,4.5,48.2,13.7
Cincinnati Bengals,460,5.9,21,7,337,14,7.2,4.0,42.6,10.0
Los Angeles Rams,460,6.0,23,5,355,18,7.3,4.0,45.9,12.2
Green Bay Packers,450,5.8,13,6,375,7,6.9,4.3,44.5,6.4
San Francisco 49ers,427,6.1,24,10,361,14,7.7,4.3,41.2,11.5
Tennessee Titans,419,5.1,25,11,362,14,5.9,4.4,38.9,13.0"""

csv_21_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Buffalo Bills,289,4.6,30,11,285,19,4.8,4.2,28.0,14.5
Tampa Bay Buccaneers,353,5.2,29,12,344,17,5.6,4.3,33.5,14.7
Tennessee Titans,354,5.4,22,6,331,16,6.2,3.9,37.1,10.8
Kansas City Chiefs,364,5.9,29,14,363,15,6.7,4.8,37.3,16.0
San Francisco 49ers,365,5.1,20,11,318,9,5.9,4.0,34.4,9.3
Green Bay Packers,371,5.4,26,8,337,18,5.8,4.7,36.6,14.0
Los Angeles Rams,372,5.2,25,6,347,19,6.1,4.0,35.9,13.3
Cincinnati Bengals,376,5.5,21,8,339,13,6.3,4.3,33.9,11.1"""

csv_22_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Kansas City Chiefs,496,6.4,23,11,408,12,7.5,4.7,46.4,10.1
Philadelphia Eagles,477,5.9,19,10,385,9,7.1,4.6,42.5,10.2
Dallas Cowboys,467,5.4,23,5,347,18,6.4,4.3,41.5,10.8
Buffalo Bills,455,6.1,27,13,367,14,6.8,5.2,45.0,15.2
San Francisco 49ers,450,5.9,17,8,347,9,7.1,4.7,41.4,9.1
Cincinnati Bengals,418,5.5,18,6,357,12,6.5,3.8,41.5,9.7
Jacksonville Jaguars,404,5.7,22,13,352,9,6.3,4.7,39.2,12.2
New York Giants,365,5.2,16,10,352,6,5.5,4.8,37.3,7.3"""

csv_22_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
San Francisco 49ers,277,5.0,30,10,291,20,5.9,3.4,25.7,15.3
Buffalo Bills,286,5.1,27,10,313,17,5.6,4.3,31.0,14.3
Cincinnati Bengals,322,5.4,24,11,295,13,6.2,4.2,36.4,13.1
Dallas Cowboys,342,5.1,33,17,327,16,5.7,4.4,33.5,16.2
Philadelphia Eagles,344,4.8,27,10,325,17,4.9,4.6,32.4,15.1
Jacksonville Jaguars,350,5.4,27,13,350,14,6.4,4.2,36.1,14.4
Kansas City Chiefs,369,5.1,20,9,337,11,5.6,4.4,35.9,10.3
New York Giants,371,5.6,19,13,370,6,6.0,5.2,39.8,8.8"""

csv_23_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
San Francisco 49ers,491,6.6,18,6,383,12,8.4,4.8,45.3,10.1
Baltimore Ravens,483,5.9,19,12,360,7,6.8,4.9,43.1,9.6
Detroit Lions,461,5.9,23,11,375,12,6.9,4.6,40.6,11.8
Buffalo Bills,451,5.7,28,10,381,18,6.9,4.3,41.4,14.9
Green Bay Packers,383,5.6,18,7,338,11,6.5,4.3,39.7,9.2
Houston Texans,377,5.4,14,6,329,8,6.5,3.7,37.2,7.3
Kansas City Chiefs,371,5.5,28,11,350,17,6.3,4.3,39.3,14.6
Tampa Bay Buccaneers,348,5.1,18,8,300,10,6.3,3.4,35.5,9.8"""

csv_23_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Baltimore Ravens,280,4.6,31,13,306,18,4.7,4.5,28.7,14.4
Kansas City Chiefs,294,4.7,17,9,300,8,4.9,4.5,28.5,9.5
San Francisco 49ers,298,5.0,28,6,319,22,5.4,4.1,33.9,15.5
Buffalo Bills,311,5.1,30,12,313,18,5.5,4.6,32.0,16.0
Tampa Bay Buccaneers,325,5.4,26,13,337,13,6.4,3.8,34.4,14.2
Green Bay Packers,350,5.4,18,11,340,7,6.2,4.4,38.7,9.8
Houston Texans,353,5.2,24,10,332,14,6.5,3.5,34.7,11.1
Detroit Lions,395,5.5,23,7,339,16,6.7,3.7,37.3,12.4"""

csv_24_offense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PF,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Detroit Lions,564,6.2,15,3,410,12,7.7,4.7,51.6,8.2
Buffalo Bills,525,6.0,8,2,360,6,7.3,4.5,49.7,4.6
Baltimore Ravens,518,6.8,11,7,393,4,8.1,5.8,47.0,6.1
Washington Commanders,485,5.7,16,7,380,9,6.4,5.0,50.0,8.0
Philadelphia Eagles,463,5.6,15,9,360,6,6.5,4.9,44.5,6.6
Kansas City Chiefs,385,5.1,14,3,350,11,5.9,4.0,43.2,8.3
Houston Texans,372,5.1,19,7,308,12,5.7,4.4,36.8,9.3
Los Angeles Rams,367,5.4,14,5,342,9,6.6,3.9,39.1,8.3"""

csv_24_defense = """,,Tot,Tot,,,Pass,Pass,Rush,,
Tm,PA,Y/P,TO,FL,1stD,Int,NY/A,Y/A,Sc%,TO%
Philadelphia Eagles,303,4.7,26,13,293,13,5.1,4.3,31.6,14.1
Kansas City Chiefs,326,5.3,20,7,328,13,6.1,4.1,36.5,11.4
Detroit Lions,342,5.7,24,8,331,16,6.4,4.5,35.4,13.3
Baltimore Ravens,361,5.2,17,5,330,12,6.1,3.6,36.6,9.1
Buffalo Bills,368,5.5,32,16,356,16,6.2,4.5,36.3,17.5
Houston Texans,372,5.1,29,10,313,19,5.7,4.3,35.2,14.0
Los Angeles Rams,386,5.7,20,7,342,13,6.5,4.6,42.9,10.6
Washington Commanders,391,5.4,17,10,335,7,6.0,4.8,41.5,8.8"""

## Data Preprocessing

Now we have to make some changes to these tables. First we need to rename the columns to appropriate names that also don't overlap between offense and defense as we will want to combine these two eventually. Additionally, we want the year for a given team's results to also be stored, so we're adding a column for that.

This step creates dataframes for the offense and defense stats of each year.

In [405]:
# Only run once, if you need to run again, clear variables first (below). this is because of all the dataframes we are creating

year_count = 0
off_def = True   # We alternate between halves of the for-loop in order to correctly name a dataframe offensive or defensive
csv_files = [csv_00_offense, csv_00_defense, csv_01_offense, csv_01_defense, csv_02_offense, csv_02_defense, csv_03_offense, csv_03_defense, csv_04_offense, csv_04_defense, csv_05_offense, csv_05_defense, csv_06_offense, csv_06_defense, csv_07_offense, csv_07_defense, csv_08_offense, csv_08_defense, csv_09_offense, csv_09_defense, csv_10_offense, csv_10_defense, csv_11_offense, csv_11_defense, csv_12_offense, csv_12_defense, csv_13_offense, csv_13_defense, csv_14_offense, csv_14_defense, csv_15_offense, csv_15_defense, csv_16_offense, csv_16_defense, csv_17_offense, csv_17_defense, csv_18_offense, csv_18_defense, csv_19_offense, csv_19_defense, csv_20_offense, csv_20_defense, csv_21_offense, csv_21_defense, csv_22_offense, csv_22_defense, csv_23_offense, csv_23_defense, csv_24_offense, csv_24_defense]

for f in csv_files:
    temp_df = pd.read_csv(io.StringIO(f))
    temp_df = temp_df.iloc[1:]   # Delete the first row (column names written in the incorrect spot)

    if off_def == True:
        temp_df.rename(columns={'Unnamed: 0': 'Team', 'Unnamed: 1': 'Points_scored', 'Tot': 'Y/P', 'Tot.1': 'Turnovers', 'Unnamed: 4': 'Fumbles', 'Unnamed: 5': '1stD', 'Pass': 'Interceptions_allowed', 'Pass.1': 'NY/A', 'Rush': 'Y/A', 'Unnamed: 9': 'Sc%_offense', 'Unnamed: 10': 'TO%_offense'}, inplace=True)
        df_name = f'offense_{year_count}'
        globals()[df_name] = temp_df
        off_def = False
        continue
        
    if off_def == False:   # Same as about but for defensive data
        temp_df.rename(columns={'Unnamed: 0': 'Team', 'Unnamed: 1': 'Points_allowed', 'Tot': 'Y/P_allowed', 'Tot.1': 'Takeaways', 'Unnamed: 4': 'Fumbles_lost', 'Unnamed: 5': '1stD_allowed', 'Pass': 'Interceptions', 'Pass.1': 'NY/a_Allowed', 'Rush': 'Y/A_allowed', 'Unnamed: 9': 'Sc%_defense', 'Unnamed: 10': 'TO%_defense'}, inplace=True)
        df_name = f'defense_{year_count}'
        globals()[df_name] = temp_df
        year_count += 1
        off_def = True

In [406]:
# delete all variables
# %reset -f

In [407]:
# Now we're adding a year column to the dataframes
dfs = [offense_0, defense_0, offense_1, defense_1, offense_2, defense_2, offense_3, defense_3, offense_4, defense_4, offense_5, defense_5, offense_6, defense_6, offense_7, defense_7, offense_8, defense_8, offense_9, defense_9, offense_10, defense_10, offense_11, defense_11, offense_12, defense_12, offense_13, defense_13, offense_14, defense_14, offense_15, defense_15, offense_16, defense_16, offense_17, defense_17, offense_18, defense_18, offense_19, defense_19, offense_20, defense_20, offense_21, defense_21, offense_22, defense_22, offense_23, defense_23, offense_24, defense_24]
year_count = 2000
off_def = True
for index in range(len(dfs)):
    dfs[index]['Year'] = [year_count, year_count, year_count, year_count, year_count, year_count, year_count, year_count]
    if off_def == True:
        off_def = False
        continue
    if off_def == False:
        year_count += 1   # So that we're not incrementing the year with each dataset, as the first two are 2000, next two 2001, and so on (offense and defense)
        off_def = True

In [408]:
# Combining the offensive and defensive data based on a team in a given year
# Again, if you need to run this twice, you might need to reset the variables
year = 2000
for index in range(0, len(dfs)-1, 2):
    temp_df = pd.merge(dfs[index], dfs[index+1], on=['Team', 'Year'])
    df_name = f'df_{year}'
    globals()[df_name] = temp_df
    year += 1

## Adding the Rankings (to be predicted)

In [409]:
# Looking at each dataframe, we're going to manually assign rankings to each team, for each year
df_2023

,Team,Points_scored,Y/P,Turnovers,Fumbles,1stD,Interceptions_allowed,NY/A,Y/A,Sc%_offense,...,Points_allowed,Y/P_allowed,Takeaways,Fumbles_lost,1stD_allowed,Interceptions,NY/a_Allowed,Y/A_allowed,Sc%_defense,TO%_defense
0,San Francisco 49ers,491,6.6,18,6,383,12,8.4,4.8,45.3,...,298,5.0,28,6,319,22,5.4,4.1,33.9,15.5
1,Baltimore Ravens,483,5.9,19,12,360,7,6.8,4.9,43.1,...,280,4.6,31,13,306,18,4.7,4.5,28.7,14.4
2,Detroit Lions,461,5.9,23,11,375,12,6.9,4.6,40.6,...,395,5.5,23,7,339,16,6.7,3.7,37.3,12.4
3,Buffalo Bills,451,5.7,28,10,381,18,6.9,4.3,41.4,...,311,5.1,30,12,313,18,5.5,4.6,32.0,16.0
4,Green Bay Packers,383,5.6,18,7,338,11,6.5,4.3,39.7,...,350,5.4,18,11,340,7,6.2,4.4,38.7,9.8
5,Houston Texans,377,5.4,14,6,329,8,6.5,3.7,37.2,...,353,5.2,24,10,332,14,6.5,3.5,34.7,11.1
6,Kansas City Chiefs,371,5.5,28,11,350,17,6.3,4.3,39.3,...,294,4.7,17,9,300,8,4.9,4.5,28.5,9.5
7,Tampa Bay Buccaneers,348,5.1,18,8,300,10,6.3,3.4,35.5,...,325,5.4,26,13,337,13,6.4,3.8,34.4,14.2


In [410]:
# This is all just based on a quick google search into each years playoff games

# 2023
rank = [2, 4, 3, 5, 7, 8, 1, 6]
df_2023['Rank'] = rank

# 2022
rank = [1, 2, 6, 7, 4, 3, 5, 8]
df_2022['Rank'] = rank

# 2021
rank = [6, 5, 3, 2, 1, 8, 4, 7]
df_2021['Rank'] = rank

# 2020
rank = [3, 4, 1, 5, 2, 8, 7, 6]
df_2020['Rank'] = rank

# 2019
rank = [7, 2, 1, 8, 6, 3, 5, 4]
df_2019['Rank'] = rank

# 2018
rank = [3, 2, 4, 1, 8, 5, 7, 6]
df_2018['Rank'] = rank

# 2017
rank = [2, 1, 6, 3, 5, 4, 8, 7]
df_2017['Rank'] = rank

# 2016
rank = [1, 3, 5, 2, 4, 8, 6, 7]
df_2016['Rank'] = rank

# 2015
rank = [2, 4, 3, 5, 8, 7, 6, 1]
df_2015['Rank'] = rank

# 2014
rank = [3, 8, 1, 6, 4, 5, 2, 7]
df_2014['Rank'] = rank

# 2013
rank = [2, 4, 1, 7, 3, 6, 5, 8]
df_2013['Rank'] = rank

# 2012
rank = [4, 5, 6, 3, 7, 8, 1, 2]
df_2012['Rank'] = rank

# 2011
rank = [6, 5, 2, 1, 7, 4, 3, 8]
df_2011['Rank'] = rank

# 2010
rank = [7, 8, 1, 2, 3, 5, 4, 6]
df_2010['Rank'] = rank

# 2009
rank = [1, 3, 5, 2, 8, 6, 7, 4]
df_2009['Rank'] = rank

# 2008
rank = [5, 7, 2, 3, 6, 4, 8, 1]
df_2008['Rank'] = rank

# 2007
rank = [2, 8, 5, 3, 4, 6, 7, 1]
df_2007['Rank'] = rank

# 2006
rank = [7, 2, 1, 4, 5, 3, 8, 6]
df_2006['Rank'] = rank

# 2005
rank = [2, 6, 3, 4, 1, 7, 8, 5]
df_2005['Rank'] = rank

# 2004
rank = [8, 1, 7, 2, 3, 4, 6, 5]
df_2004['Rank'] = rank

# 2003
rank = [5, 6, 3, 7, 8, 4, 1, 2]
df_2003['Rank'] = rank

# 2002
rank = [2, 4, 8, 5, 7, 3, 6, 1]
df_2002['Rank'] = rank

# 2001
rank = [2, 7, 6, 1, 4, 3, 5, 8]
df_2001['Rank'] = rank

# 2000
rank = [3, 4, 5, 7, 6, 1, 2, 8]
df_2000['Rank'] = rank

In [411]:
# Finally, combine all the individual dataframes into a large one
dfs = [df_2000, df_2001, df_2002, df_2003, df_2004, df_2005, df_2006, df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023]
df = pd.concat(dfs)
df

,Team,Points_scored,Y/P,Turnovers,Fumbles,1stD,Interceptions_allowed,NY/A,Y/A,Sc%_offense,...,Y/P_allowed,Takeaways,Fumbles_lost,1stD_allowed,Interceptions,NY/a_Allowed,Y/A_allowed,Sc%_defense,TO%_defense,Rank
0,Oakland Raiders,479,5.6,20,9,337,11,6.6,4.8,39.2,...,5.2,37,16,320,21,5.9,4.0,26.9,18.7,3
1,Minnesota Vikings,397,6.2,28,10,319,18,7.2,5.0,39.7,...,5.6,18,10,344,8,6.4,4.5,38.2,9.4,4
2,New Orleans Saints,354,5.2,26,11,312,15,6.2,4.1,32.4,...,5.0,35,15,279,20,5.5,4.2,27.7,17.5,5
3,Philadelphia Eagles,351,4.9,29,14,295,15,5.0,4.7,33.3,...,4.6,31,12,295,19,5.0,4.0,25.1,14.9,7
4,Tennessee Titans,346,5.2,30,14,299,16,6.7,3.8,30.4,...,4.2,30,13,215,17,4.7,3.6,18.1,13.8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,Buffalo Bills,451,5.7,28,10,381,18,6.9,4.3,41.4,...,5.1,30,12,313,18,5.5,4.6,32.0,16.0,5
4,Green Bay Packers,383,5.6,18,7,338,11,6.5,4.3,39.7,...,5.4,18,11,340,7,6.2,4.4,38.7,9.8,7
5,Houston Texans,377,5.4,14,6,329,8,6.5,3.7,37.2,...,5.2,24,10,332,14,6.5,3.5,34.7,11.1,8
6,Kansas City Chiefs,371,5.5,28,11,350,17,6.3,4.3,39.3,...,4.7,17,9,300,8,4.9,4.5,28.5,9.5,1


## Machine Learning (Training)

In [412]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [413]:
# Our first pass will be a basic linear regression model

X = df.drop(columns=['Team', 'Rank'])   # The team itself shouldn't influence our predictions, solely their performance
y = df['Rank']   # We're predicting the rankings

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Multi_class so that we can predict the rankings of each individual team
# Class_weight so that importance isn't given to teams of a given ranking, allows for more even training
model = LogisticRegression(max_iter=1000, solver='liblinear', multi_class='ovr', class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 23.08%


AHH! 23.08%!!! That's quite low. Even though a random set of guesses would give me roughly 12.5% which is worse, we should aim for higher.

There are a couple of important points to note here.
1. We're not working with a lot of data. Over 24 years, 8 teams per year, that's only 192 rows of data. For as simplified a model as this, we would need a far larger dataset
2. Logistic regression does not perform well with too many features, and in this case we have about 20
3. We have not scaled our data, which is important since we're working with all kinds of numbers
4. For example, total points scored by a team can be in the 500s, whereas something like yards per rushing attempt could be say 4.6

What's another model we can try?

Let's try a random forest regressor. Now we don't have a lot of data for the model to accurately classify each team with a ranking, so instead we'll allow the model to assign a continous value to each team as its ranking. Then, going in ascending order of values, we can determine the outcomes of the remaining games.

In [415]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [418]:
# Random Forest Regressor

X = df.drop(columns=['Team', 'Rank'])
y = df['Rank']

# Standardize the features, for less variable training (features on similar scales)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create and train the model (100 trees, the random state ensures the same results, which is helpful while debugging)
model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_scaled, y)

rmse_scores = np.sqrt(-cross_val_score(model, X_scaled, y, scoring='neg_mean_squared_error', cv=5)) # AI helped

# Calculate the range of the target variable
y_range = y.max() - y.min()

# Calculate normalized accuracy
normalized_accuracy = (1 - (np.mean(rmse_scores) / y_range)) * 100   # AI

print(f"\nNormalized Accuracy: {normalized_accuracy:.2f}%")   # Normalized since we scaled our data


Normalized Accuracy: 67.05%


That's much better than our first try! 

## Feature Importance

Now I wonder which features have been the most valuable, or essentially the most correlated to rank?

In [419]:
# Feature importance

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)


Feature Importance:
                  Feature  Importance
0           Points_scored    0.089362
8             Sc%_offense    0.081894
19            Sc%_defense    0.076986
4                    1stD    0.074493
18            Y/A_allowed    0.055754
7                     Y/A    0.052613
6                    NY/A    0.050483
11         Points_allowed    0.049626
17           NY/a_Allowed    0.047045
12            Y/P_allowed    0.043707
3                 Fumbles    0.043481
15           1stD_allowed    0.042003
20            TO%_defense    0.040731
9             TO%_offense    0.038062
16          Interceptions    0.037660
14           Fumbles_lost    0.034940
13              Takeaways    0.032850
10                   Year    0.032020
5   Interceptions_allowed    0.029276
1                     Y/P    0.027267
2               Turnovers    0.019747


Looks like the top features were points scored by a team, percentage of drives ending in an offensive score, how many of those a team's defense allowed, 1st downs, and rushing yards allowed per attempt. This only further shows how crucial a good performance from both the offense and the defense teams is in ensuring a teams overall success.

## Final Predictions

The fun part! Using the model we trained above, let's now make some predictions for these last 7 games of the 2024-2025 NFL season.

In [420]:
df_2024

,Team,Points_scored,Y/P,Turnovers,Fumbles,1stD,Interceptions_allowed,NY/A,Y/A,Sc%_offense,...,Points_allowed,Y/P_allowed,Takeaways,Fumbles_lost,1stD_allowed,Interceptions,NY/a_Allowed,Y/A_allowed,Sc%_defense,TO%_defense
0,Detroit Lions,564,6.2,15,3,410,12,7.7,4.7,51.6,...,342,5.7,24,8,331,16,6.4,4.5,35.4,13.3
1,Buffalo Bills,525,6.0,8,2,360,6,7.3,4.5,49.7,...,368,5.5,32,16,356,16,6.2,4.5,36.3,17.5
2,Baltimore Ravens,518,6.8,11,7,393,4,8.1,5.8,47.0,...,361,5.2,17,5,330,12,6.1,3.6,36.6,9.1
3,Washington Commanders,485,5.7,16,7,380,9,6.4,5.0,50.0,...,391,5.4,17,10,335,7,6.0,4.8,41.5,8.8
4,Philadelphia Eagles,463,5.6,15,9,360,6,6.5,4.9,44.5,...,303,4.7,26,13,293,13,5.1,4.3,31.6,14.1
5,Kansas City Chiefs,385,5.1,14,3,350,11,5.9,4.0,43.2,...,326,5.3,20,7,328,13,6.1,4.1,36.5,11.4
6,Houston Texans,372,5.1,19,7,308,12,5.7,4.4,36.8,...,372,5.1,29,10,313,19,5.7,4.3,35.2,14.0
7,Los Angeles Rams,367,5.4,14,5,342,9,6.6,3.9,39.1,...,386,5.7,20,7,342,13,6.5,4.6,42.9,10.6


In [421]:
new_data = df_2024.drop(columns=['Team'])   # We don't include rank here unlike before because that's what we're predicting

new_data_scaled = scaler.transform(new_data)   # Standardize the new data using the same scaler

predictions = model.predict(new_data_scaled)
print("Predictions:", predictions)

Predictions: [3.08 3.91 4.34 4.26 4.04 4.94 6.25 6.34]


In [422]:
results = []
for i in range(len(predictions)):
    team, rank = df_2024.iloc[i, 0], predictions[i]
    results.append((team, rank))
results = sorted(results, key=lambda x: x[1])
print(results)

[('Detroit Lions', 3.08), ('Buffalo Bills', 3.91), ('Philadelphia Eagles', 4.04), ('Washington Commanders', 4.26), ('Baltimore Ravens', 4.34), ('Kansas City Chiefs', 4.94), ('Houston Texans', 6.25), ('Los Angeles Rams', 6.34)]


### Finally, based on the above rankings, we can create the following, final predictions:

#### Divisional Round:
##### Chiefs beat the Texans
##### Bills beat the Ravens
##### Lions beat the Commanders
##### Eagles beat the Rams

#### Conference Championships:
##### Bills beat the Chiefs
##### Lions beat the Eagles

#### Superbowl:
##### **Lions** beat the Bills !!!

*Note: this doesn't follow the original ranking system, but that's something to take into consideration for next year

## Conclusion

We've reached the end of our analysis! As a final note, some improvements/adjustments to be made for next year:
1. Try to find a larger dataset to work with, for a higher accuracy
2. Fine-tune the random forest model further, to see where it can improve
3. Potentially include more head to head data between two teams, rather than just their overall performances